**Решение тестового для вакансии https://hh.ru/vacancy/48973633?from=favorite_vacancy_list**

## Задание №1

> В специальной лотерее игрок выбирает 5 разных чисел из возможных 30. В этот же день случайным образом определяется победная комбинация из 7 разных чисел (из того же набора). Игрок считается победителем в случае, если совпадают любые 4 числа в выбранном им наборе и в победной комбинации.
> 

Какова вероятность выигрыша?

Число выигрышных сочетаний:

In [12]:
7*6*5*4*23*22

425040

Число всего сочетаний

In [13]:
30*29*28*27*26

17100720

Вероятность этого события:

In [14]:
round(425040/17100720 * 100,2)

2.49

## Задание №2

> В файле "Lightmap. Тестовое задание №2.csv" построчно приведены записи событий установки приложения (install) и начала сессии (login). В столбце player_id указан уникальный id пользователя, в столбце event_timestamp - время события (unix timestamp UTC). 

Удержание первого дня определяется как доля уникальных пользователей, отправивших событие login на следующий *календарный* день, среди всех пользователей, отправивших событие install в текущий *календарный* день. Например, если 1 сентября 100 пользователей скачали приложение, и 50 *из них* совершили логин 2 сентября, удержание первого дня для 1 сентября составляет 50%.

Удержание первого дня для периода дат определяется как отношение общего числа уникальных пользователей, вернувшихся в приложение на следующий день после дня установки, к общему числу уникальных пользователей, совершивших установку в рамках заданного периода дат.
>
Например, если 2 сентября 200 пользователей скачали приложение, и 75 из них совершили логин 3 сентября, удержание первого дня за период 1-2 сентября составляет: (50 + 75) / (100 + 200) ≈ 42%




Вопрос: Можно ли с уверенностью утверждать, что удержание первого дня за период с 15 по 18 сентября выше, чем удержание первого дня за период с 19 по 22 сентября? И почему?

In [1]:
!pip install plotly==5.2.1

     |████████████████████████████████| 21.8 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
df1 = pd.read_csv('/gdrive/MyDrive/lightmap/Lightmap. Тестовое задание №2.csv')

In [5]:
df1

,player_id,event_timestamp,event_name
0,1058930,1632265733,install
1,1055681,1632412526,login
2,1057784,1632411775,login
3,1015250,1631923078,login
4,1040648,1631670785,login
...,...,...,...
52538,1060090,1632338669,install
52539,1061249,1632403477,install
52540,1062028,1632436754,install
52541,1050142,1631918819,login


Переведем столбец timestamp во время, ограничивая данные только датой.

In [6]:
df1['time'] = [datetime.utcfromtimestamp(int(el)).strftime('%Y-%m-%d')
              for el in df1.event_timestamp
              ]


Таким образом, получаем:

In [7]:
df1.head()

,player_id,event_timestamp,event_name,time
0,1058930,1632265733,install,2021-09-21
1,1055681,1632412526,login,2021-09-23
2,1057784,1632411775,login,2021-09-23
3,1015250,1631923078,login,2021-09-17
4,1040648,1631670785,login,2021-09-15


Создадим сводную таблицу, в которой посчитаем количество уникальных пользователей, которые логинились или устанавливали приложение, для каждого из дней.

In [9]:
table = df1.pivot_table(index=['time','event_name'], values='player_id', aggfunc='nunique').reset_index()
table

,time,event_name,player_id
0,2021-09-15,install,1327
1,2021-09-15,login,2026
2,2021-09-16,install,1284
3,2021-09-16,login,2098
4,2021-09-17,install,1483
5,2021-09-17,login,2286
6,2021-09-18,install,1779
7,2021-09-18,login,2645
8,2021-09-19,install,1674
9,2021-09-19,login,2725


Удержание первого дня за период с 15 по 18 сентября:

In [10]:
install_15_18 = table[(table['event_name'] == 'install') & 
                    (table['time'] < '2021-09-19')].player_id.sum()
login_15_18 = table[(table['event_name'] == 'login') & 
                    (table['time'] < '2021-09-20') & 
                    (table['time'] > '2021-09-15')].player_id.sum()
print(round(install_15_18 / login_15_18 * 100, 2))

60.21


Удержание первого дня за период с 19 по 22 сентября:

In [11]:
install_19_22 = table[(table['event_name'] == 'install') & 
                    (table['time'] > '2021-09-18') & 
                    (table['time'] < '2021-09-23')].player_id.sum()
login_20_23 = table[(table['event_name'] == 'login') & 
                    (table['time'] > '2021-09-19')].player_id.sum()
print(round(install_19_22 / login_20_23 * 100, 2))

65.36


**Вопрос:** 

Можно ли с уверенностью утверждать, что удержание первого дня за период с 15 по 18 сентября выше, чем удержание первого дня за период с 19 по 22 сентября? И почему?

**Ответ:**
Удержание первого дня за период с 19 по 22 сентября выше, чем удержание первого дня за период с 15 по 18 сентября. 

## Задание №3

> В файле "Lightmap. Тестовое задание №3.csv" построчно приведены записи транзакций, совершенных в приложении когортой пользователей. В столбце event_timestamp указано время транзакции (unix timestamp UTC), в столбце registration_timestamp - дата регистрации пользователя, в столбце inapp_size - размер соответствующего платежа (сумма $).
> 

Таблица 

[Lightmap. Тестовое задание №3.csv](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/18c44675-33f0-4f3c-8ad7-f98f52f57bfe/Lightmap.___3.csv)

Требуется построить кривую накопительной суммы платежей когорты по дням с момента регистрации (один день - одна точка на кривой). На основании полученных данных построить и отобразить в виде другой кривой на том же графике прогноз значений накопительной суммы для дней жизни с 90 по 180.

In [ ]:
df2 = pd.read_csv('/gdrive/MyDrive/lightmap/Lightmap. Тестовое задание №3.csv')

In [ ]:
df2

,event_timestamp,registration_timestamp,inapp_size
0,1611154038,1610971359,5
1,1611759898,1610972522,3
2,1611057594,1610974545,5
3,1611562555,1610976251,3
4,1611082290,1610976461,3
...,...,...,...
11572,1611556376,1610970090,3
11573,1611556310,1610970090,3
11574,1611592979,1610970090,1
11575,1611116708,1610970537,3


Переведем оба столбца с timestamp в дату. Также сразу переведем полученные столбцы в формат datetime.

In [ ]:
df2['time_event'] = [datetime.utcfromtimestamp(int(el)).strftime('%Y-%m-%d')
              for el in df2.event_timestamp
              ]
df2['time_reg'] = [datetime.utcfromtimestamp(int(el)).strftime('%Y-%m-%d')
              for el in df2.registration_timestamp
              ]
df2['time_event'] = pd.to_datetime(df2['time_event'])
df2['time_reg'] = pd.to_datetime(df2['time_reg'])

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11577 entries, 0 to 11576
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_timestamp         11577 non-null  int64         
 1   registration_timestamp  11577 non-null  int64         
 2   inapp_size              11577 non-null  int64         
 3   time_event              11577 non-null  datetime64[ns]
 4   time_reg                11577 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(3)
memory usage: 452.4 KB


Создадим столбец разности времени регистрации и ивента.

In [ ]:
df2['count'] = df2['time_event'] - df2['time_reg']

Построим сводную таблицу для суммы платежей для каждого из дней с момента регистрации.

In [ ]:
table2 = df2.pivot_table(index='count', values='inapp_size', 
                        aggfunc='sum').reset_index()
table2

,count,inapp_size
0,0 days,18491
1,1 days,6933
2,2 days,4341
3,3 days,3397
4,4 days,2978
...,...,...
86,86 days,286
87,87 days,467
88,88 days,300
89,89 days,299


In [ ]:
table2['days'] = table2['count'].dt.days

Создадим столбец накопительной суммы для платежей когорты по дням с момента регистрации:

In [ ]:
table2['cumsum'] = table2.inapp_size.cumsum()

Чтоб можно было построить логарифмическую трендовую линию, уберем из рассмотрения 0 день.

In [ ]:
table2_0 = table2[table2['days'] > 0]

Построим кривую накопительной суммы платежей когорты по дням с момента регистрации (один день - одна точка на кривой). Для прогноза значений накопительной суммы для дней жизни с 90 по 180 построим логарифмическую регрессию.

In [ ]:
px.scatter(table2_0, x='days', y="cumsum", trendline="ols", trendline_options=dict(log_x=True))

Уравнение для полученной регрессии, представлено на графике выше и имеет вид:

In [ ]:
ols_reg = lambda days: 31925.5 * np.log10(days)+14544.9

In [ ]:
y = {x: ols_reg(x)
     for x in range(1,181)}

На основании полученных данных построим совместно имеющийся график распределения накопительной суммы и в виде другой кривой на том же графике прогноз значений накопительной суммы для дней жизни с 90 по 180.

In [ ]:
go.Figure(data=[go.Scatter(name='ols_reg', x=list(y.keys()), y = list(y.values())), 
                go.Scatter(name='real_values', x=table2_0['days'], y=table2_0["cumsum"])])
